In [ ]:

import pandas as pd
import numpy as np

cols = ['ID', 'Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'N-Score', 'E-Score', 'O-Score', 'A-Score', 'C-Score', 'Impulsive', 'ImpSS', 'Alcohol', 'Amphet',
        'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA']

df = pd.read_csv('./data/drug_consumption.data', names=cols)
df.pop('ID')
df.head()
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import accuracy_score, classification_report

# # Assuming the dataset is loaded as df
# x = df.iloc[:, 1:12].values
# y = df.iloc[:, 12:].values

# # Splitting the dataset
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# # Using OneVsRest with RandomForest
# clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=100))
# clf.fit(x_train, y_train)

# # Making predictions
# y_pred = clf.predict(x_test)

# # Evaluating the model
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Given columns
cols = ['ID', 'Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'N-Score', 'E-Score', 'O-Score', 'A-Score', 'C-Score', 'Impulsive', 'ImpSS', 'Alcohol', 'Amphet',
        'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA']

# Load data
df = pd.read_csv('./data/drug_consumption.data', names=cols)
df.pop('ID')

# Define feature columns and target columns
feature_cols = df.columns[:12] # Assuming the first 12 columns are features
target_cols = df.columns[12:]  # Assuming the drug columns are targets

# Convert target columns to binary (1 for use, 0 for no use)
for col in target_cols:
    df[col] = df[col].apply(lambda x: 1 if x != 'CL0' else 0)  # Assuming 'CL0' denotes non-use

# Split data
X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df[target_cols], test_size=0.2)

# Initialize and train a random forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on test data
y_pred = clf.predict(X_test)

# Evaluate performance
for idx, col in enumerate(target_cols):
    print(f"Report for {col}:")
    print(classification_report(y_test.iloc[:, idx], y_pred[:, idx], zero_division=1))
    print("-------------")
